<a href="https://colab.research.google.com/github/praveentn/hgwxx7/blob/master/deeplearning/rnn/keras_rnn_lstm_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import load_model
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking
from keras.optimizers import Adam
from keras.utils import Sequence
from keras.preprocessing.text import Tokenizer

from sklearn.utils import shuffle

from IPython.display import HTML

from itertools import chain
from keras.utils import plot_model
import numpy as np
import pandas as pd
import random
import json
import re

RANDOM_STATE = 50
TRAIN_FRACTION = 0.7

Using TensorFlow backend.


In [0]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML

InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
warnings.filterwarnings('ignore', category = UserWarning)

import pandas as pd
import numpy as np

import utils
from utils import get_data, get_embeddings, find_closest

In [3]:
data = pd.read_csv('TPB_ML_Amadeus_Industry_62.csv', encoding='latin')
data.head()

,IndustryCode,TEText
0,6211,(a) acquisition tenure administration and disp...
1,62121,1 construction facilities and maintenance. 2 t...
2,62121,1 trade to the wholesale and the retail. Comme...
3,6209,1. computer telecommunications and Office Auto...
4,62121,1. other professional scientific and technical...


In [4]:
training_dict, word_idx, idx_word, sequences = get_data('TPB_ML_Amadeus_Industry_62.csv', training_len = 50)

There are 1574 unique words.
There are 310 sequences.


In [5]:
training_dict['X_train'][:2]
training_dict['y_train'][:2]

array([[ 278,    1,   68,    3,  158,    1, 1278, 1279, 1280,  539, 1281,
          56,  540, 1282,   67,  541, 1283,  542,  345, 1284,  263, 1285,
         543,    1, 1286,  211, 1287,  324,   19,  211,   10,  462,  539,
         544,  540,  545, 1288, 1289,  211, 1290,    1, 1291,   42,  542,
           1,   34,  211,   70,   76,    2],
       [ 738,   17,  304,   16,  739,  411,  740,  741,  742,  412,  307,
         110,   17,  743,  744,  745,   18,   74,   16,  746,   17,   51,
         185,   44,   18,  747,   61,   18,  748,    2,   18,  230,  749,
         413,  750,  395,  414,  751,  752,  753,  754,  755,  756,  757,
         127,   61,  758,   17,  759,   18]])

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [6]:
for i, sequence in enumerate(training_dict['X_train'][:2]):
    text = []
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text) + '\n')
    print('Label: ' + idx_word[np.argmax(training_dict['y_train'][i])] + '\n')

Features: import and sale of children and adult clothes shoes accessories toys training articles fitness equipment housewares kitchen foods interior decoration furniture books children's and baby items gifts skin care items , cosmetics accessories cleaning articles textiles knives outdoor items knitted and crocheted products foods and other items compatible therewith .

Label: hiring

Features: SERVICE AND MAINTENANCE OF WEBSITES SEO (SEARCH ENGI OPTIMIZATION) WEB MARKETING MANAGEMENT AND CALL EC ER THE PROVISION OF MATERIALS AND SERVICES RELATED TO THE ABOVE IN THE INVITE . THE COMPANY CAN ALSO CARRY OUT FINANCIAL OPERATIONS ERCIALI INDUSTRIAL CO TU TTE (SO NOT IN PUBLIC AND EXCLUSIVELY THE

Label: LOSING



In [0]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam

from keras.utils import plot_model

In [8]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=len(word_idx) + 1,
        output_dim=300,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(len(word_idx) + 1, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         472200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1574)              102310    
Total params: 672,110
Trainable params: 672,110
Non-trainable params: 0
_______________

In [19]:
h = model.fit(training_dict['X_train'], training_dict['y_train'], epochs=300, batch_size=1024, 
          validation_data = (training_dict['X_valid'], training_dict['y_valid']), verbose=1)

Train on 217 samples, validate on 93 samples
Epoch 1/300
217/217 [==============================] - 0s 710us/step - loss: 0.4440 - acc: 0.9124 - val_loss: 12.9816 - val_acc: 0.0108
Epoch 2/300
217/217 [==============================] - 0s 702us/step - loss: 0.4209 - acc: 0.9032 - val_loss: 12.9844 - val_acc: 0.0108
Epoch 3/300
217/217 [==============================] - 0s 620us/step - loss: 0.4544 - acc: 0.8710 - val_loss: 12.9858 - val_acc: 0.0108
Epoch 4/300
217/217 [==============================] - 0s 605us/step - loss: 0.4931 - acc: 0.8940 - val_loss: 12.9873 - val_acc: 0.0108
Epoch 5/300
217/217 [==============================] - 0s 608us/step - loss: 0.4517 - acc: 0.8848 - val_loss: 12.9896 - val_acc: 0.0108
Epoch 6/300
217/217 [==============================] - 0s 624us/step - loss: 0.4221 - acc: 0.8940 - val_loss: 12.9905 - val_acc: 0.0108
Epoch 7/300
217/217 [==============================] - 0s 621us/step - loss: 0.3722 - acc: 0.9171 - val_loss: 12.9912 - val_acc: 0.0108
Epo

In [20]:
print('Model Performance: Log Loss and Accuracy on training data')
model.evaluate(training_dict['X_train'], training_dict['y_train'], batch_size = 2048)

print('\nModel Performance: Log Loss and Accuracy on validation data')
model.evaluate(training_dict['X_valid'], training_dict['y_valid'], batch_size = 2048)

Model Performance: Log Loss and Accuracy on training data
217/217 [==============================] - 0s 135us/step


[0.05222279950976372, 0.9953917264938354]


Model Performance: Log Loss and Accuracy on validation data
93/93 [==============================] - 0s 335us/step


[13.635411262512207, 0.01075268816202879]

In [21]:
embeddings = get_embeddings(model)
embeddings.shape

(1574, 300)

In [22]:
find_closest('agriculture', embeddings, word_idx, idx_word)


Query: agriculture

Word: agriculture     Cosine Similarity: 1.0
Word: legal           Cosine Similarity: 0.2092999964952469
Word: lease           Cosine Similarity: 0.1932000070810318
Word: INDIVIDUALS     Cosine Similarity: 0.1906999945640564
Word: NFRONTI         Cosine Similarity: 0.18070000410079956
Word: ORDER           Cosine Similarity: 0.17470000684261322
Word: RELATED         Cosine Similarity: 0.1737000048160553
Word: 17              Cosine Similarity: 0.16619999706745148
Word: exercise        Cosine Similarity: 0.16279999911785126
Word: MEDICAL         Cosine Similarity: 0.16269999742507935


In [23]:
find_closest('brokerage', embeddings, word_idx, idx_word)


Query: brokerage

Word: brokerage       Cosine Similarity: 1.0
Word: ESPECIALLY      Cosine Similarity: 0.21930000185966492
Word: customers       Cosine Similarity: 0.1809999942779541
Word: lectures        Cosine Similarity: 0.1729000061750412
Word: shall           Cosine Similarity: 0.1720999926328659
Word: Consultant      Cosine Similarity: 0.14990000426769257
Word: day             Cosine Similarity: 0.14720000326633453
Word: evidence        Cosine Similarity: 0.14309999346733093
Word: individuals     Cosine Similarity: 0.14159999787807465
Word: photographic    Cosine Similarity: 0.14010000228881836


In [25]:
find_closest('computers', embeddings, word_idx, idx_word)


Query: computers

Word: computers       Cosine Similarity: 1.0
Word: Web             Cosine Similarity: 0.19140000641345978
Word: tenure          Cosine Similarity: 0.1826999932527542
Word: bucondenal      Cosine Similarity: 0.17720000445842743
Word: shoot           Cosine Similarity: 0.1720000058412552
Word: papers          Cosine Similarity: 0.15710000693798065
Word: legisl          Cosine Similarity: 0.15440000593662262
Word: certification   Cosine Similarity: 0.14839999377727509
Word: Code            Cosine Similarity: 0.14569999277591705
Word: patterns        Cosine Similarity: 0.14249999821186066
